**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4034.10 - Análisis de grandes volúmenes de datos**

Tecnológico de Monterrey

* Dr. Iván Olmos Pineda
* Mtra. Verónica Sandra Guzmán de Valle
* Mtro. Alberto Daniel Salinas Montemayor

**Proyecto** | Base de Datos de Big Data

---

**Equipo 37**

|  NOMBRE COMPLETO                        |     MATRÍCULA     | 
| :-------------------------------------: |:-----------------:|
| Alejandro Díaz Villagómez               |  A01276769        | 
| Alonso Pedrero Martínez                 |  A01769076        | 
| César Iván Pedrero Martínez             |  A01366501        | 
| Emiliano Saucedo Arriola                |  A01659258        | 

## Intro

In [20]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, lit, countDistinct
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, FloatType, DecimalType, ShortType, NumericType

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

In [2]:
findspark.init()
print(findspark.find())

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

/Users/alejandrodiazvillagomez/Desktop/Proyecto-Big-Data-PySpark/.venv/lib/python3.12/site-packages/pyspark


25/04/30 21:05:29 WARN Utils: Your hostname, Alejandros-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.196 instead (on interface en0)
25/04/30 21:05:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 21:05:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import kagglehub

# Download latest version
FILE_PATH = kagglehub.dataset_download("machharavikiran/amazon-reviews")

print("Path to dataset files:", FILE_PATH)

/Users/alejandrodiazvillagomez/Desktop/Proyecto-Big-Data-PySpark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/alejandrodiazvillagomez/.cache/kagglehub/datasets/machharavikiran/amazon-reviews/versions/1


In [11]:
df_reviews = spark.read.csv(
    FILE_PATH,
    header=True,
    inferSchema=True,
    multiLine=True,
    escape="\"",
    quote="\""
)

print("Number of records in the dataset:", df_reviews.count())
print("Number of columns in the dataset:", len(df_reviews.columns))
df_reviews.limit(10).toPandas()
# df_reviews.limit(10)

Number of records in the dataset: 6906564
Number of columns in the dataset: 16


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,22873041,R3ARRMDEGED8RD,B00KJWQIIC,335625766,Plemo 14-Inch Laptop Sleeve Case Waterproof Fa...,PC,5,0,0,N,Y,Pleasantly surprised,I was very surprised at the high quality of th...,2015-08-31,1
1,US,30088427,RQ28TSA020Y6J,B013ALA9LA,671157305,TP-Link OnHub AC1900 Wireless Wi-Fi Router,PC,5,24,31,N,N,OnHub is a pretty no nonsense type router that...,I am a Google employee and had to chance to us...,2015-08-31,1
2,US,20329786,RUXJRZCT6953M,B00PML2GQ8,982036237,AmazonBasics USB 3.0 A Male to A Male Cable - ...,PC,1,2,2,N,N,None of them worked. No functionality at all.,"Bought cables in 3ft, 6ft and 9ft. NONE of th...",2015-08-31,0
3,US,14215710,R7EO0UO6BPB71,B001NS0OZ4,576587596,Transcend P8 15-in-1 USB 2.0 Flash Memory Card...,PC,1,0,0,N,Y,just keep searching.,"nope, cheap and slow",2015-08-31,0
4,US,38264512,R39NJY2YJ1JFSV,B00AQMTND2,964759214,Aleratec SATA Data Cable 2.0 20in Serial ATA S...,PC,5,0,0,N,Y,Five Stars,Excellent! Great value and does the job.,2015-08-31,1
5,US,30548466,R31SR7REWNX7CF,B00KX4TORI,170101802,Kingston Digital MobileLite G4 USB 3.0 Multi-F...,PC,5,0,0,N,Y,"Good quality, works well and compact","Good quality,works well and compact size",2015-08-31,1
6,US,589298,RVBP8I1R0CTZ8,B00P17WEMY,206124740,White 9 Inch Unlocked Dual Sim Card Phone Tabl...,PC,3,1,2,N,Y,in fact this is third China good. Demn s***,This demn tablet is just a Real Chinese produc...,2015-08-31,0
7,US,49329488,R1QF6RS1PDLU18,B00TR05L9Y,778403103,"Lenovo TAB2 A10 - 10.1"" 2-in-1 Tablet (1.5Ghz,...",PC,4,1,1,N,Y,Good,I am not sure I don't know if it is the tablet...,2015-08-31,1
8,US,50728290,R23AICGEDAJQL1,B0098Y77OG,177098042,Acer,PC,1,0,0,N,Y,You get what you pay for,"After exactly 45 days, the screen went dark. P...",2015-08-31,0
9,US,37802374,R2EY3N4K9W19UP,B00IFYEYXC,602496520,AzureWave Broadcom BCM94352HMB 802.11/ac/867Mb...,PC,5,3,4,N,Y,Great for Windows 7 Laptop!,Replaced my Intel Centrino 2230 with the BCM94...,2015-08-31,1


## Caracterización de la población

In [23]:
def schema_information(df_input):
    """
    This method shows the current schema of the data.
    """
    df_input.printSchema()


def missing_values_table(df_input):
    """
    Displays a table with the count of missing values per column.
    """
    missing_exprs = []
    
    for c in df_input.schema.fields:
        field_name = c.name
        field_type = c.dataType
        
        if isinstance(field_type, (IntegerType, LongType, DoubleType, FloatType, DecimalType, ShortType, NumericType)):
            missing_exprs.append(
                count(when(col(field_name).isNull() | isnan(col(field_name)), field_name)).alias(field_name)
            )
        elif isinstance(field_type, StringType):
            missing_exprs.append(
                count(when(col(field_name).isNull() | (col(field_name) == ""), field_name)).alias(field_name)
            )
        else:
            missing_exprs.append(
                count(when(col(field_name).isNull(), field_name)).alias(field_name)
            )

    df_missing_values = df_input.select(missing_exprs)
    df_missing_values = df_missing_values.toPandas().T.reset_index()
    df_missing_values.columns = ["Column Name", "Missing Values"]
    return df_missing_values


def unique_values_table(df_input):
    """
    Displays a table with the count of unique values per column.
    """
    unique_exprs = []
    
    for c in df_input.schema.fields:
        field_name = c.name
        field_type = c.dataType
        
        if isinstance(field_type, (IntegerType, LongType, DoubleType, FloatType, DecimalType, ShortType, NumericType)):
            unique_exprs.append(
                countDistinct(col(field_name)).alias(field_name)
            )
        elif isinstance(field_type, StringType):
            unique_exprs.append(
                countDistinct(when(col(field_name) != "", col(field_name))).alias(field_name)
            )
        else:
            unique_exprs.append(
                countDistinct(col(field_name)).alias(field_name)
            )

    df_unique_values = df_input.select(unique_exprs)
    df_unique_values = df_unique_values.toPandas().T.reset_index()
    df_unique_values.columns = ["Column Name", "Unique Values"]
    return df_unique_values


def descriptive_statistics(df_input, variable_type="numeric"):
    """
    Shows descriptive statistics for the dataset.

    Parameters:
    - df_input: Spark DataFrame
    - variable_type: 'numeric' to show numeric columns, 'non-numeric' to show others
    """
    numeric_cols = [field.name for field in df_input.schema.fields if isinstance(field.dataType, NumericType)]
    non_numeric_cols = [field.name for field in df_input.schema.fields if field.name not in numeric_cols]

    if variable_type == "numeric":
        if numeric_cols:
            display(df_input.select(numeric_cols).summary().toPandas())
        else:
            print("No numeric columns found.")
    
    elif variable_type == "non-numeric":
        if non_numeric_cols:
            for col_name in non_numeric_cols:
                print(f"Columna: {col_name}")
                print("--" * 20)
                display(df_input.groupBy(col(col_name)).count().orderBy("count", ascending=False).limit(5).toPandas())
        else:
            print("No non-numeric columns found.")
    
    else:
        print(f"Invalid variable_type: {variable_type}. Choose 'numeric' or 'non-numeric'.")

In [ ]:
print("=== Schema Information ===")
schema_information(df_reviews)


=== Schema Information ===
root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- sentiment: integer (nullable = true)



In [24]:
print("=== Missing Values Table ===")
df_missing_values = missing_values_table(df_reviews)
df_missing_values.columns = ["Column Name", "Missing Values"]
df_missing_values

=== Missing Values Table ===


,Column Name,Missing Values
0,marketplace,0
1,customer_id,0
2,review_id,0
3,product_id,0
4,product_parent,0
5,product_title,0
6,product_category,0
7,star_rating,0
8,helpful_votes,0
9,total_votes,0


In [ ]:
print("=== Unique Values Table ===")
df_unique_values = unique_values_table(df_reviews)
df_unique_values

=== Unique Values Table ===


,index,0
0,marketplace,1
1,customer_id,4055429
2,review_id,6906564
3,product_id,441903
4,product_parent,382279
5,product_title,385803
6,product_category,1
7,star_rating,5
8,helpful_votes,1169
9,total_votes,1216


In [18]:
print("=== Descriptive Statistics - Numeric Columns ===")
descriptive_statistics(df_reviews, variable_type="numeric")

=== Descriptive Statistics - Numeric Columns ===


,summary,customer_id,product_parent,star_rating,helpful_votes,total_votes,sentiment
0,count,6906564,6906564,6906564,6906564,6906564,6906564
1,mean,2.799257247625172E7,4.9166335483220977E8,4.086460937739808,1.4844511684826203,1.9620770907212328,0.7636061868101128
2,stddev,1.5572197138978068E7,2.8901115508420634E8,1.362854001234071,41.49142556564166,43.07079218821245,0.424866807848865
3,min,10016,1602,1,0,0,0
4,25%,14419422,242889999,4,0,0,1
5,50%,26791807,488442804,5,0,0,1
6,75%,42659658,742866380,5,1,1,1
7,max,53096585,999996025,5,47524,48362,1


In [19]:
print("=== Descriptive Statistics - Non-numeric Columns ===")
descriptive_statistics(df_reviews, variable_type="non-numeric")

=== Descriptive Statistics - Non-numeric Columns ===
Columna: marketplace
----------------------------------------


,marketplace,count
0,US,6906564


Columna: review_id
----------------------------------------


,review_id,count
0,R10005OK073QCX,1
1,R16ELHIDT9GRP9,1
2,R100069LQE9ZU9,1
3,R1JVXTBV6IGY55,1
4,R1000RMPYUVD9E,1


Columna: product_id
----------------------------------------


...

,review_headline,count
0,Five Stars,814873
1,Four Stars,158973
2,One Star,80618
3,Three Stars,59598
4,Two Stars,33963


Columna: review_body
----------------------------------------


,review_body,count
0,good,12612
1,Good,11224
2,Great,9584
3,Excellent,7995
4,Works great,6732


Columna: review_date
----------------------------------------


,review_date,count
0,2015-01-05,16067
1,2015-01-04,15720
2,2015-01-03,13931
3,2015-01-07,13375
4,2014-12-29,12523


## Particionamiento

## Técnica de muestreo a aplicar por partición